# Sisal Selenium Scraper Test

This notebook tests the Selenium-based scraper with a specific live match: **Francia U21 vs Georgia U21**.

The scraper will connect to the Sisal page, extract betting odds, and create a BettingOdds instance.

In [2]:
import sys
import os

# Add the project root to Python path
# Get the current notebook directory and go up one level to reach project root
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else 'scraper_test.ipynb'))
project_root = os.path.dirname(current_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import the Selenium scraper
from src.scraper.sisal_selenium_scraper import scrape_sisal_odds_selenium

In [ ]:
# Test scraper with Francia U21 vs Georgia U21 match
from src.scraper.sisal_selenium_scraper import scrape_sisal_odds_selenium

# Target match URL
url = "https://www.sisal.it/scommesse-live/evento/calcio/euro-u21/francia-u21-georgia-u21"

print("Testing Sisal Selenium Scraper")
print(f"Match: Francia U21 vs Georgia U21")
print(f"URL: {url}")
print("=" * 60)

# Scrape the odds
odds = scrape_sisal_odds_selenium(url, headless=True)

if odds:
    print("✅ SUCCESS! Betting odds extracted")
    print(f"Match: {odds.home_team} vs {odds.away_team}")
    print(f"Timestamp: {odds.timestamp}")
    print(f"Source: {odds.source}")
    print()
    
    # Display odds
    print("Extracted odds:")
    print(f"  1X2: {odds.home_win} / {odds.draw} / {odds.away_win}")
    print(f"  O/U 2.5: {odds.over_2_5} / {odds.under_2_5}")
    print(f"  BTTS: {odds.both_teams_score_yes} / {odds.both_teams_score_no}")
    print(f"  Double Chance: {odds.home_or_draw} / {odds.away_or_draw} / {odds.home_or_away}")
    print()
    
    # Show complete BettingOdds object
    print("Complete BettingOdds object:")
    print(repr(odds))
    
else:
    print("❌ Failed to extract odds")
    print("Possible reasons:")
    print("- Match not live")
    print("- Geographic restrictions") 
    print("- Page structure changed")

=== Testing Specific Match: Francia U21 vs Georgia U21 ===
URL: https://www.sisal.it/scommesse-live/evento/calcio/euro-u21/francia-u21-georgia-u21
Starting scraping process...
Note: This may take 30-60 seconds due to browser automation
🎯 Starting Selenium scraping for: https://www.sisal.it/scommesse-live/evento/calcio/euro-u21/francia-u21-georgia-u21
✅ Chrome WebDriver setup successful
🌐 Simulating realistic browser behavior...
📄 Visiting Sisal homepage...
⏱️ Simulating reading time: 4.6s
🎲 Navigating to live betting...
⏱️ Simulating reading time: 2.8s
